In [3]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



count of reviews: 6189


In [4]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
print(doc_clean[:100])

['simply awesome else say', 'contrary site t-mobile lead believe t-mobile could provide coverage home address stop amazon.com selling phone t-mobile transferring existing working amp number service course took two week gt 10 phone call .after receiving reception called t-mobile informed actually cover area.i called amp back happily attempted restore account find t-mobile refused release numbers.after month essentially coverage anyone phone happily back amp belong.the bad news t-mobile sent bill double quoted monthly rate amazon.com refunded purchase price.even area could get coverage phone work well pressing key screen would lock needed remove battery reset it.last warning watch charge inbound message -- really dislike t-mobile subscriber send lot text message free site -- subscriber pay 0.05/message', "'ve t610 3 month 'm still love highly recommend getting bluetooth adapter computer take advantage customizing baby 've downloaded midi file game photo web copied t610 free sm charge web

In [5]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
print(tagged[:100])

[('simply', 'RB'), ('awesome', 'JJ'), ('else', 'RB'), ('say', 'VBP'), ('contrary', 'JJ'), ('site', 'NN'), ('t-mobile', 'JJ'), ('lead', 'JJ'), ('believe', 'VBP'), ('t-mobile', 'NN'), ('could', 'MD'), ('provide', 'VB'), ('coverage', 'NN'), ('home', 'NN'), ('address', 'NN'), ('stop', 'JJ'), ('amazon.com', 'NN'), ('selling', 'VBG'), ('phone', 'NN'), ('t-mobile', 'JJ'), ('transferring', 'VBG'), ('existing', 'VBG'), ('working', 'VBG'), ('amp', 'JJ'), ('number', 'NN'), ('service', 'NN'), ('course', 'NN'), ('took', 'VBD'), ('two', 'CD'), ('week', 'NN'), ('gt', 'VBD'), ('10', 'CD'), ('phone', 'NN'), ('call', 'NN'), ('.after', 'NN'), ('receiving', 'VBG'), ('reception', 'NN'), ('called', 'VBN'), ('t-mobile', 'NN'), ('informed', 'VBN'), ('actually', 'RB'), ('cover', 'VB'), ('area.i', 'NN'), ('called', 'VBN'), ('amp', 'NN'), ('back', 'RB'), ('happily', 'RB'), ('attempted', 'VBN'), ('restore', 'NN'), ('account', 'NN'), ('find', 'VBP'), ('t-mobile', 'NN'), ('refused', 'VBD'), ('release', 'NN'), ('num

In [6]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
print(noun_phrase_tagged[:100])
#doc_complete = nouns

[('simply', 'RB'), ('awesome', 'JJ'), ('else', 'RB'), ('say', 'VBP'), Tree('NOUN_PHRASE', [('contrary', 'JJ'), ('site', 'NN'), ('t-mobile', 'JJ'), ('lead', 'JJ')]), ('believe', 'VBP'), Tree('NOUN_PHRASE', [('t-mobile', 'NN')]), ('could', 'MD'), ('provide', 'VB'), Tree('NOUN_PHRASE', [('coverage', 'NN'), ('home', 'NN'), ('address', 'NN'), ('stop', 'JJ'), ('amazon.com', 'NN')]), ('selling', 'VBG'), Tree('NOUN_PHRASE', [('phone', 'NN'), ('t-mobile', 'JJ')]), ('transferring', 'VBG'), ('existing', 'VBG'), ('working', 'VBG'), Tree('NOUN_PHRASE', [('amp', 'JJ'), ('number', 'NN'), ('service', 'NN'), ('course', 'NN'), ('took', 'VBD'), ('two', 'CD'), ('week', 'NN')]), ('gt', 'VBD'), Tree('NOUN_PHRASE', [('10', 'CD'), ('phone', 'NN'), ('call', 'NN'), ('.after', 'NN'), ('receiving', 'VBG'), ('reception', 'NN')]), ('called', 'VBN'), Tree('NOUN_PHRASE', [('t-mobile', 'NN'), ('informed', 'VBN'), ('actually', 'RB')]), ('cover', 'VB'), Tree('NOUN_PHRASE', [('area.i', 'NN'), ('called', 'VBN'), ('amp', '

In [7]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
print(nouns[:100])
doc_complete = nouns

['site', 't-mobile', 'coverage', 'home', 'address', 'amazon.com', 'phone', 'number', 'service', 'course', 'week', 'phone', 'call', '.after', 'reception', 't-mobile', 'area.i', 'amp', 'restore', 'account', 't-mobile', 'release', 'month', 'coverage', 'anyone', 'phone', 'belong.the', 'news', 'sent', 'bill', 'rate', 'amazon.com', 'purchase', 'area', 'coverage', 'phone', 'work', 'screen', 'battery', 'reset', 'warning', 'watch', 'charge', 'message', 'send', 'lot', 'message', 'site', 'pay', 'month', 'computer', 'advantage', 'baby', 'midi', 'file', 'game', 'photo', 'web', 'sm', 'charge', 'site', 'fee', 'everything', 'cell', 'phone', 'camera', 'color', 'display', 'battery', 'life', 'standby', 'day', 'hour', 'side', 'color', 'display', 'see', 'daylight', 'time', 'antenna', 'reception', 'bar', 'bldgs', 'look', 't630', 'color', 'display', 'think', 'investment', 'mobile', 'phone', 'thing', 'cell', 'phone', 'race', 'phone', 'best', 'pricetag', 'reception', 'piece']


In [8]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



[('phone', 31300), ('battery', 4819), ('screen', 4159), ('use', 3855), ('time', 3594), ('camera', 3393), ('work', 3322), ('day', 2752), ('device', 2584), ('iphone', 2581), ('thing', 2516), ('feature', 2300), ('life', 2149), ('problem', 2114), ('quality', 1948), ('samsung', 1894), ('call', 1849), ('apps', 1741), ('card', 1740), ('way', 1685), ('year', 1591), ('lot', 1521), ('button', 1511), ('video', 1494), ('service', 1465), ('issue', 1456), ('htc', 1427), ('case', 1416), ('month', 1402), ('data', 1400), ('price', 1382), ('android', 1363), ('galaxy', 1331), ('note', 1321), ('review', 1311), ('get', 1304), ('google', 1298), ('people', 1283), ('music', 1282), ('picture', 1281), ('size', 1206), ('everything', 1176), ('keyboard', 1160), ('version', 1137), ('software', 1125), ('hour', 1110), ('app', 1088), ('look', 1068), ('hand', 1057), ('memory', 1029), ('plan', 1004), ('network', 996), ('verizon', 994), ('home', 977), ('charge', 960), ('apple', 956), ('something', 955), ('minute', 897), 

def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [9]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

['10 phone call .after receiving reception', 'essentially coverage anyone phone happily back amp belong.the']
phone has absolute avg sentiment 0.21881028636805386
['great battery life still good standby', 't-mobile complaint weak battery']
battery has absolute avg sentiment 0.21924825217215724
['46.53 amazon return pretty color screen', 'software capability 65k color screen']
screen has absolute avg sentiment 0.25582457822766813
['use phone even 10 minute one day', 'like use speaker type hand free kit car']
use has absolute avg sentiment 0.2244972323879237
['talk time cell phone 840min', 'talk time 5 hour']
time has absolute avg sentiment 0.1659173913043481
['5 star following reasons:1 camera', 'work without hitch camera']
camera has absolute avg sentiment 0.2666072094468631
['work without hitch camera', 'phone wireless modem internet connectivity work easygprs software provided manage phone entry']
work has absolute avg sentiment 0.2550370100273478
["last 2 3 day stand n't", 'use phon

apple has absolute avg sentiment 0.2042761546723952
['fool self.people review something clue', 'camera/video something always']
something has absolute avg sentiment 0.1747514705882351
['regular phone use 10 minute day', 'quot t-zones quot minute two month']
minute has absolute avg sentiment 0.11855450145208102
['antenna internal basically resemble blackberry outside much sought feature treo user', '3rd party program almost good blackberry']
blackberry has absolute avg sentiment 0.21058249516441
['tungsten design connector requires separate power supply charge', 'laptop extra power connection messy annoyance upside since battery']
power has absolute avg sentiment 0.1861512367491165
['sim contact transfered phone memory dice scroll speed', 'speed']
speed has absolute avg sentiment 0.1930304730473045
['inexpensively enough people priced smartphone market', 'normal usage look online.the smartphone market']
smartphone has absolute avg sentiment 0.212647307286167
['another free navigational 

In [10]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [11]:
features=  [ {"feature" : key} for (key,val) in dict_noun_abs_sentiment.items() if val >0.01]
print(features)

[{'feature': 'phone'}, {'feature': 'battery'}, {'feature': 'screen'}, {'feature': 'use'}, {'feature': 'time'}, {'feature': 'camera'}, {'feature': 'work'}, {'feature': 'day'}, {'feature': 'device'}, {'feature': 'iphone'}, {'feature': 'thing'}, {'feature': 'feature'}, {'feature': 'life'}, {'feature': 'problem'}, {'feature': 'quality'}, {'feature': 'samsung'}, {'feature': 'call'}, {'feature': 'apps'}, {'feature': 'card'}, {'feature': 'way'}, {'feature': 'year'}, {'feature': 'lot'}, {'feature': 'button'}, {'feature': 'video'}, {'feature': 'service'}, {'feature': 'issue'}, {'feature': 'htc'}, {'feature': 'case'}, {'feature': 'month'}, {'feature': 'data'}, {'feature': 'price'}, {'feature': 'android'}, {'feature': 'galaxy'}, {'feature': 'note'}, {'feature': 'review'}, {'feature': 'get'}, {'feature': 'google'}, {'feature': 'people'}, {'feature': 'music'}, {'feature': 'picture'}, {'feature': 'size'}, {'feature': 'everything'}, {'feature': 'keyboard'}, {'feature': 'version'}, {'feature': 'softwa

In [12]:
my_collection.insert_many(features)